In [ ]:
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,Input
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from PIL import Image

In [ ]:
TRAIN_DIR = 'img_dfds\Train'
TEST_DIR = 'img_dfds\Validation'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

Fake completed
Real completed


In [5]:
train.head()

,image,label
0,img_dfds\Train\Fake\fake_0.jpg,Fake
1,img_dfds\Train\Fake\fake_1.jpg,Fake
2,img_dfds\Train\Fake\fake_10.jpg,Fake
3,img_dfds\Train\Fake\fake_100.jpg,Fake
4,img_dfds\Train\Fake\fake_1000.jpg,Fake


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

Fake completed
Real completed


In [7]:
test.head()

,image,label
0,img_dfds\Validation\Fake\fake_0.jpg,Fake
1,img_dfds\Validation\Fake\fake_1.jpg,Fake
2,img_dfds\Validation\Fake\fake_10.jpg,Fake
3,img_dfds\Validation\Fake\fake_100.jpg,Fake
4,img_dfds\Validation\Fake\fake_1000.jpg,Fake


In [8]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode="grayscale")
        new_size = (48, 48)
        img = img.resize(new_size)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [9]:
train_features = extract_features(train['image'])

  0%|          | 0/140002 [00:00<?, ?it/s]

In [10]:
test_features = extract_features(test['image'])

  0%|          | 0/39428 [00:00<?, ?it/s]

In [11]:
x_train = train_features/255.0
x_test = test_features/255.0

In [12]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [13]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [14]:
y_train = to_categorical(y_train,num_classes = 2)
y_test = to_categorical(y_test,num_classes = 2)

In [15]:
model = Sequential()
model.add(Input(shape=(48, 48, 1)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(2, activation='softmax'))

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 4, validation_data = (x_test,y_test))

Epoch 1/4
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 1205s 1s/step - accuracy: 0.6948 - loss: 0.5490 - val_accuracy: 0.8372 - val_loss: 0.3535
Epoch 2/4
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 1229s 1s/step - accuracy: 0.8453 - loss: 0.3483 - val_accuracy: 0.8690 - val_loss: 0.2949
Epoch 3/4
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 1248s 1s/step - accuracy: 0.8691 - loss: 0.3048 - val_accuracy: 0.8745 - val_loss: 0.2849
Epoch 4/4
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 1254s 1s/step - accuracy: 0.8833 - loss: 0.2745 - val_accuracy: 0.8977 - val_loss: 0.2428


In [20]:
model_json = model.to_json()
with open("deepfake2.json",'w') as json_file:
    json_file.write(model_json)
model.save("deepfake2.keras")